In [18]:
#!/usr/bin/env python
# coding: utf-8

# # Import Libraries

import pandas as pd
import os
from matplotlib import pyplot as plt
import math
from tqdm import tqdm
from copy import copy
import random
import warnings
import numpy as np
warnings.simplefilter(
    action='ignore', category=FutureWarning)  # FutureWarning 제거
from time import time
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

# # Define Util Function 

def vol_merge(truck_df):
    '''
    트럭의 물량을 합포장하는 함수
    '''
    unique_cds = truck_df['SPG_INNB'].unique()
    SPG_INNB_lst = []
    x_lst = []
    y_lst = []
    reassign_lst = []
    _assign_lst = []
    KRRI_REC_S_lst = []
    dong_lst = []
    deliver_lst = []
    volume_lst = []
    iter_lst = []
    
    for idx, code in tqdm(enumerate(unique_cds)):
        SPG_INNB_lst.append(truck_df[truck_df['SPG_INNB'] == code]['SPG_INNB'].values[0])
        x_lst.append(truck_df[truck_df['SPG_INNB'] == code]['x'].values[0])
        y_lst.append(truck_df[truck_df['SPG_INNB'] == code]['y'].values[0])
        reassign_lst.append(truck_df[truck_df['SPG_INNB'] == code]['reassign'].values[0])
        _assign_lst.append(truck_df[truck_df['SPG_INNB'] == code]['assign'].values[0])
        KRRI_REC_S_lst.append(truck_df[truck_df['SPG_INNB'] == code]['KRRI_REC_S'].values[0])
        dong_lst.append(truck_df[truck_df['SPG_INNB'] == code]['dong'].values[0])
        deliver_lst.append(truck_df[truck_df['SPG_INNB'] == code]['deliver'].values[0])
        iter_lst.append(truck_df[truck_df['SPG_INNB'] == code]['iter'].values[0])

        volume_sum = truck_df[truck_df['SPG_INNB'] == code]['volume'].sum()
        volume_lst.append(volume_sum)

        # print('-------{}th row work ahs been complete-------'.format(idx))

    res = pd.DataFrame({'SPG_INNB':SPG_INNB_lst,\
                        'x': x_lst, 'y': y_lst, 'reassign': reassign_lst, '_assign': _assign_lst, \
                        'KRRI_REG_CD': KRRI_REC_S_lst, 'dong': dong_lst, 'deliver': deliver_lst, 'volume': volume_lst, 'iter': iter_lst})
    # res.columns = [, , , , , 'KRRI_REG_CD', 'dong', 'deliver', 'volume', 'iter']

    return res

def distance(x1, y1, x2, y2):
    '''
    두 점 사이의 거리를 구하는 함수
    '''
    result = math.sqrt(math.pow(x1 - x2, 2) + math.pow(y1 - y2, 2))
    return result

def closest_membership(cl: int, new_merber_matrix):
    '''
    특정 클러스터로부터 멤버쉽밸류가 가장 큰 노드를 구하는 함수
    '''
    res = new_merber_matrix[new_merber_matrix[cl] == new_merber_matrix[cl].max()]
    return res

def closest_point(cl: int, new_dist_matrix):
    '''
    특정 클러스터를 대상으로 가장 가까운 거리의 점을 구하는 함수
    '''
    res = new_dist_matrix[new_dist_matrix[str(cl)] == new_dist_matrix[str(cl)].min()]
    return res

def furthest_point(cl: int, new_dist_matrix) -> pd.DataFrame:
    '''
    특정 클러스터를 대상으로 가장 먼 거리의 점을 구하는 함수
    '''
    res = new_dist_matrix[new_dist_matrix[str(cl)] == new_dist_matrix[str(cl)].max()]
    return res

def update_distance_matrix(mt_df, center_df) -> pd.DataFrame:
    '''
    현재 물량의 데이터와 가중 평균중심 좌표로 각 클러스터와 모든 노드 간의 거리를 업데이트 하는 함수
    '''
    dist_matrix = pd.DataFrame(index=range(0,len(mt_df)), columns = ['0','1','2','3','4','5','6','7','8','9'])
    for i in range(len(mt_df)):
        for j in range(len(center_df)):
            result = distance(mt_df.loc[i]['x'], mt_df.loc[i]['y'],center_df.loc[j]['x'],center_df.loc[j]['y'])
            dist_matrix.loc[i][j] = result
            #print(mt_df.loc[i]['x'], mt_df.loc[i]['y']) 

    for i in range(10):
        mt_df['{0}'.format(i)]  = pd.to_numeric(dist_matrix['{0}'.format(i)])
    
    return mt_df

def update_weighted_center_df(mt_df):
    '''
    현재 물량을 기준으로 클러스터 별 가중 평균 중심점을 업데이트하는 함수
    '''
    weighted_avg_x_lst = []
    weighted_avg_y_lst = []
    for i in range(10):
        temp_df = mt_df[mt_df['reassign'] == i]
        weighted_avg_x = (temp_df['x'] * temp_df['volume'] * temp_df['update_weight']).sum() / (temp_df['volume']* temp_df['update_weight']).sum()
        weighted_avg_y = (temp_df['y'] * temp_df['volume'] * temp_df['update_weight']).sum() / (temp_df['volume']* temp_df['update_weight']).sum()

        weighted_avg_x_lst.append(weighted_avg_x)
        weighted_avg_y_lst.append(weighted_avg_y)

    weighted_center = {'x': weighted_avg_x_lst,
                'y': weighted_avg_y_lst}
    weighted_center_df = pd.DataFrame(weighted_center)

    return weighted_center_df

def update_volume_lst(mt_df):
    '''
    각 클러스터별 물량을 업데이트하는 함수
    '''
    vol_lst = [0] * 10
    for cl_n in range(10):
        temp_df = mt_df[mt_df['reassign'] == cl_n]
        vol_lst[cl_n] = temp_df['volume'].sum()
    print(vol_lst)
        
    return vol_lst

def MAE(x,y):
    '''
    클러스터의 노드 교환이 끝난 시점에서 MAE를 구하는 함수
    '''
    x = np.array(x)
    y = np.array([y]*len(x))
    res = np.abs(np.subtract(x,y)).mean()
    
    return res

def min_2(lst):
    sorted_lst = sorted(lst)
    result = sorted_lst[1]
    return result

vol_dict = {}

for day in range(1,31):
    main_truck = pd.read_csv('./Information/opArea_pop_outputSegok.csv', index_col=0)
    df = pd.read_csv("./Input/main_truck_{:02d}.csv".format(day), index_col=0) ## 송장데이터를 넣는 곳 매번 바뀔 수 있음
    # 원래 02
    main_truck["volume"] = 0
    main_truck.drop(columns='idx', inplace=True)
    #각 행별로 SPG_INNB값을 셀거에요 
    #main_truck의 SPG_INNB와 Input data의 SPG_INNB가 같다면! main_truck의 volume 값을 1씩 더해줄 거에요~
    for index, row in df.iterrows():
        spg = row['SPG_INNB']
        volume = row['volume']
        main_truck.loc[main_truck['SPG_INNB'] == spg, 'volume'] += volume #df.loc[row, column], 2400개의 물량을 510개의 cell 중 어디에 해당하는지 넣는듯
    filter_cond = main_truck["volume"]!=0
    main_truck = main_truck[filter_cond]
    main_truck.reset_index(inplace=True)
    main_truck['index'] = range(len(main_truck))
    # 당일 물량 없는 cell은 지우고 나머지에 대해서만 다루는듯 
    ## master data
    ## master data rarely changes
    org_cens = pd.read_csv('./Information/output_cluster_centroids.csv', header=None) # 기존 클러스터 센트로이드 정보 -> 10개 
    mv = pd.read_csv('./Information/output_membership_value.csv', header=None) # 전체 노드의 멤버쉽 밸류 -> 510 개 
    op_segok = pd.read_csv('./Information/opArea_pop_outputSegok.csv', index_col=0) # 전체 노드의 아이디, 좌표, reassign 정보를 담고 있음
    #main_truck: 간선차량
    #df: Input(송장번호)
    org_cens.columns = ['x', 'y'] # df에 column 이름을 붙여준 것
    mv_df = pd.concat([op_segok['SPG_INNB'], mv], axis=1) # cell 고유번호와 cell의 merbership value를 옆으로 이어붙임 

    main_truck['reassign'] = 10
    new_mem_matrix = pd.merge(main_truck[['SPG_INNB', 'x', 'y', 'volume']], mv_df, how='inner') # 공통 column(SPG_INNB)를 기준으로 병합
    # [SPG_INNB, x, y, volume, merbership value]

    volume_lst = [0] * 10
    new_mem_matrix_co = copy(new_mem_matrix)
    ## 균형을 맞추면서 가까운 노드 먹어가기 -> seed decision
    while min(volume_lst) <= len(df) / 10  * 0.75: # 고정의 비율을 조정하는 라인 # df : node, 10 : cluster # 적어도 하나의 클러스터가 균등물량의 75%는 처리해야함  
        target_cl = volume_lst.index(min(volume_lst)) # min 값을 가진 index
        temp_rec = closest_membership(target_cl, new_mem_matrix_co) # 남은 cell 중 target_cl에 대해 가장 높은 membership value를 가진 행 in new_mem_martrix_co
        key = main_truck['SPG_INNB'][temp_rec.index[0]] # 위의 cell의 index 
        volume_lst[target_cl] += temp_rec['volume'].values[0] # target_cl에 temp_rec의 volume을 추가 
        main_truck.loc[main_truck['SPG_INNB'] == key, 'reassign'] = target_cl # 위의 cell의 클러스터 번호를 기존에서 target_cl로 변경 
        new_mem_matrix_co.drop([temp_rec.index[0]], inplace=True) # 이번에 사용한 cell을 제거  
        # 이거는 volume_lst 합이 2400이 안돼도 괜찮은가? 
        # 위의 while 조건 대로면 new_mem_matrix_co에 cell이 남을텐데 

    print(volume_lst)

    ## 균형 안맞추면서 가까운 노드 먹어가기 (추가)
    for _, i in enumerate(range(len(new_mem_matrix_co))):
        designate_cl = new_mem_matrix_co.iloc[i][[i for i in range(10)]].argmax()
        # print(new_mem_matrix_co.iloc[i])
        key = main_truck['SPG_INNB'][new_mem_matrix_co.index[i]]
        # print(key)
        volume_lst[designate_cl] += int(new_mem_matrix_co.iloc[i]['volume'])
        main_truck.loc[main_truck['SPG_INNB'] == key, 'reassign'] = designate_cl
    print(volume_lst)

    org_cens = pd.read_csv('./Information/output_cluster_centroids.csv', header=None) # 기존 클러스터 센트로이드 정보
    org_cens.columns = ['x', 'y']  
    print(main_truck['SPG_INNB'].duplicated().sum()) # 중복되는 집계구 없이 join되었는지 확인
    print(main_truck['volume'].sum()) # 총 물량 합 확인
    main_truck['update_weight'] = 1
    main_truck['hindrance'] = 0 # 방해 
    weighted_center_df = update_weighted_center_df(main_truck) # 가중평균 센트로이드 초기화
    # 위에서 클러스터당 최소 75%의 물량을 처리하도록 reassign 하고나면 
    main_truck_co = main_truck.copy(deep=True)
    org_cens_dist = update_distance_matrix(main_truck_co, org_cens)#main_truck.copy() # 3번째 elif문에서 쓸 org_cens와 노드간 거리 데이터
    main_truck = update_distance_matrix(main_truck, weighted_center_df)
    
    main_truck.reset_index(inplace=True)
    
    vol_lst = update_volume_lst(main_truck)
    ## update_weight를 변화시키기 위한 파라미터 설정
    u_param = 1.3
    limit_volume = 240
    ## 클러스터 물량의 표준편차 & MAE를 저장하기 위한 리스트
    truck_capa = 260
    ## 클러스터별 볼륨값 초기화
    vol_lst = update_volume_lst(main_truck)
    print('initial volume is', vol_lst)
    iteration = 0
    knn = NearestNeighbors(n_neighbors=4)
    ## 클러스터 볼륨이 260 미만이 될 때까지 작동함
    while max(vol_lst) > truck_capa:
        ## 각 클러스터에 대해서 순차적으로 균등화 작업 수행
        for i in range(10):
            ## 균등화 작업을 수행하는 클러스터 지정
            init_cluster = i
            print('\n--------------------Current init cluster {}--------------------\n'.format(init_cluster))
            ## 균등화 작업을 하는 양을 미리 지정 260과의 차이를 절대값으로 계산
            target_volume = abs(main_truck[main_truck['reassign'] == init_cluster]['volume'].sum() - 240)
            ## 점진적 작업을 위해서 작업 물량의 240과의 차이의 절반으로 정의
            target_volume /= 2
            target_volume = math.ceil(target_volume) # 자릿수 올림

            cumulative_volume = 0 # 현재 작업량 초기화
            ## 만약 작업하려는 클러스터가 물량이 over 되는 상황이라면 여기 분기로 들어옴
            ## 이 분기에서는 over되는 물량을 다른 클러스터에 나눠주게 됨
            if vol_lst[i] > limit_volume:
                cand = main_truck[main_truck['reassign'] == i] 
                for j in range(len(cand)):
                    moving_p_cand = main_truck[(main_truck['reassign'] == i) & (main_truck['ecoVehicle'] != 1)]
                    p1 = moving_p_cand[str(i)].idxmax()
                    d1_ser = moving_p_cand.loc[p1] # i번째 클러스터에서 가장 멀리있는 점(row)을 뽑음 --> 시리즈 형식
                    if d1_ser[[str(i) for i in range(10)]].values.argmin() == i: # i번째 클러스터가 가장 가까운 경우 2번째로 가까운데로 보내줘야 함
                        temp_lst = d1_ser[[str(i) for i in range(10)]].values
                        cl1 = np.argpartition(d1_ser[[str(i) for i in range(10)]].values, 1)
                        cl1 = cl1[1]
                        d1 = min_2(temp_lst)
                        print('case1-1')
                        
                    else:
                        cl1 = d1_ser[[str(i) for i in range(10)]].values.argmin()
                        d1 = d1_ser[[str(i) for i in range(10)]].values.min() # 그게 아니면 그냥 제일 가까운 클러스터(현재 속한 클러스터 X)로 보내줌
                        print('case1-2')
                                
                    d2_cand = moving_p_cand[[str(k) for k in range(10)]].min() # 가장 가까운 거리 
                    p2_cand = moving_p_cand[[str(k) for k in range(10)]].idxmin() # 내가 아닌 가장 가까운 클러스터 거리를 가진 cell 
                    dp2 = pd.concat([d2_cand,p2_cand], axis=1)
                    dp2_sorted = dp2.sort_values(by=0, ascending=True) # '0'열(d2) 기준으로 오름차순 정렬
                    if int(dp2_sorted.index[0]) == i: 
                        d2 = min_2(d2_cand)
                        p2 = int(dp2_sorted.iloc[1,1])
                        cl2 = int(dp2_sorted.index[1])
                        print('case2-1')
                    else: 
                        d2 = d2_cand.min()
                        p2 = int(dp2_sorted.iloc[0,1])
                        cl2 = int(dp2_sorted.index[0])
                        print('case2-2')
                        
                    if main_truck.iloc[p2,main_truck.columns.get_loc('hindrance')] < 5:
                        print("\nCurrent Moving Point's ID: {}".format(main_truck.iloc[p2, main_truck.columns.get_loc('SPG_INNB')]))
                        print('바뀐 클러스터는',cl2, '기존 클러스터는', main_truck.iloc[p2, main_truck.columns.get_loc('reassign')])
                        print("\nHindrance: {}".format(main_truck.iloc[p2, main_truck.columns.get_loc('hindrance')]))
                        main_truck.iloc[p2, main_truck.columns.get_loc('reassign')] = cl2
                        main_truck.iloc[p2, main_truck.columns.get_loc('update_weight')] *= u_param
                        main_truck.iloc[p2, main_truck.columns.get_loc('hindrance')] += 1
                        cumulative_volume += main_truck.iloc[p2,main_truck.columns.get_loc('volume')]
                        vol_lst = update_volume_lst(main_truck)
                        
                        print('넘친경우, if')
                    
                    elif main_truck.iloc[p2,main_truck.columns.get_loc('hindrance')] == 5:  
                        X = list(zip(moving_p_cand['x'].values, moving_p_cand['y'].values))
                        knn.fit(X)
                        temp_neighbor = knn.kneighbors(X, return_distance=False)[moving_p_cand.index.get_loc(p2)][1:] #get_loc을 왜 썼느냐? main_truck의 reassign이 i인 애들만 가져오다 보니 인덱스가 그대로 쓸수없고 X가 리스트라 인덱스가 예를들어 270인 애가 저 리스트에선 몇번째인지 알기위해서
                        dist_lst = []
                        for j in range(temp_neighbor.shape[0]):
                            temp_idx = moving_p_cand.iloc[temp_neighbor[j],main_truck.columns.get_loc('index')]
                            dist_lst.append([temp_idx, org_cens_dist.loc[temp_idx,'{}'.format(init_cluster)]])
                        dist_lst = sorted(dist_lst, key=lambda x: x[1], reverse=True)
                        for j in range(2):
                            if main_truck.loc[dist_lst[j][0],'hindrance'] < 5:
                                if main_truck.loc[dist_lst[j][0],:][[str(k) for k in range(10)]].idxmin() == str(init_cluster):
                                    cl2 = main_truck.loc[dist_lst[j][0],:][[str(k) for k in range(10)]].sort_values().index[1]
                                else:
                                    cl2 = main_truck.loc[dist_lst[j][0],:][[str(k) for k in range(10)]].sort_values().index[0]
                                main_truck.loc[dist_lst[j][0], 'reassign'] = cl2
                                vol_lst = update_volume_lst(main_truck)
                                cumulative_volume += main_truck.loc[dist_lst[j][0], 'volume']
                                main_truck.iloc[dist_lst[j][0], main_truck.columns.get_loc('update_weight')] *= u_param
                                print('Target Volume: {0}\t Current Cummulative Volume: {1}'.format(target_volume, cumulative_volume))
                                print('d2의 knn조건으로 moving_p이동 \n cluster: {0} -> {1}'.format(init_cluster, cl2))
                            else: 
                                print("\nCurrent Moving Point's ID: {}".format(main_truck.iloc[p1, main_truck.columns.get_loc('SPG_INNB')]))
                                print('바뀐 클러스터는',cl1, '기존 클러스터는', main_truck.iloc[p1, main_truck.columns.get_loc('reassign')])
                                print("\nHindrance: {}".format(main_truck.iloc[p1, main_truck.columns.get_loc('hindrance')]))
                                main_truck.iloc[p1, main_truck.columns.get_loc('reassign')] = cl1
                                main_truck.iloc[p1, main_truck.columns.get_loc('update_weight')] *= u_param
                                cumulative_volume += d1_ser['volume']
                                vol_lst = update_volume_lst(main_truck)
                            if cumulative_volume >= target_volume:
                                weighted_center_df = update_weighted_center_df(main_truck)
                                main_truck = update_distance_matrix(main_truck, weighted_center_df)
                                print('init_cluster{} has been done'.format(init_cluster))
                                break
          
                    print('Target Volume: {0}\t Current Cummulative Volume: {1}'.format(target_volume, cumulative_volume))
                    if cumulative_volume >= target_volume:
                        weighted_center_df = update_weighted_center_df(main_truck)
                        main_truck = update_distance_matrix(main_truck, weighted_center_df)
                        print('init_cluster{} has been done'.format(init_cluster))
                        break
                if max(vol_lst) <= 260:
                    break
            ## 이 분기에서는 short되는 물량을 다른 클러스터에서 가져오게 됨
            elif vol_lst[i] < limit_volume:
                ## 현재 작업이 수행 중인 클러스터에 속하지 않는 포인트들을
                ## 가장 가까이 있는 점부터 정렬하여
                ## 이동시킬 포인트들의 후보를 저장
                moving_p_cand = main_truck[(main_truck['reassign'] != i) & (main_truck['ecoVehicle'] != 1)].sort_values(by=[str(i)], ascending=True)
                for j in range(len(moving_p_cand)):
                    moving_p = moving_p_cand.iloc[j]
                    print("\nCurrent Moving Point's ID: {}".format(moving_p['SPG_INNB']))
                    print("Current Moving Point's cluster: {}".format(moving_p['reassign'])) # 추가적으로 이동 포인트의 기존 클러스터 정보를 출력
                    print("Current Moving Point's Volume: {}".format(moving_p['volume']))
                    temp_idx = moving_p['index']
                    # main_truck.iloc[temp_idx, main_truck.columns.get_loc('reassign')] = init_cluster # 현재 이동포인트를 작업 중인 클러스터로 편입
                    # main_truck.iloc[temp_idx, main_truck.columns.get_loc('update_weight')] *= u_param
                    # 이동 포인트에서 가장 거리가 가까운 클러스터가 현재 작업 중인 클러스터가 맞을 경우
                    if moving_p[[str(i) for i in range(10)]].values.argmin() == init_cluster:
                        main_truck.iloc[temp_idx, main_truck.columns.get_loc('reassign')] = init_cluster # 현재 이동포인트를 작업 중인 클러스터로 편입
                        main_truck.iloc[temp_idx, main_truck.columns.get_loc('update_weight')] *= u_param
                    ## 이동 포인트에서 가장 거리가 가까운 클러스터가 현재 작업 중인 클러스터가 맞을 경우(보통의 경우, 원래 해당하는 클러스터에 가까움)
                    else:
                        ## 2번째로 가까운 클러스터에 편입시킴 -> 이럴 경우 작업 중인 클러스터가 잡힘
                        indice = np.argpartition(moving_p[[str(i) for i in range(10)]].values,1)
                        target_cluster = indice[1]
                        if target_cluster == init_cluster:
                            main_truck.iloc[temp_idx, main_truck.columns.get_loc('reassign')] = target_cluster
                            main_truck.iloc[temp_idx, main_truck.columns.get_loc('update_weight')] *= u_param
                        elif target_cluster != init_cluster:
                            print('!!!!!!taget and init is different!!!!!!')
                            continue

                    cumulative_volume += moving_p['volume']
                    print('Target Volume: {0}\t Current Cummulative Volume: {1}'.format(target_volume, cumulative_volume))
                    vol_lst = update_volume_lst(main_truck)

                    if cumulative_volume >= target_volume:
                        weighted_center_df = update_weighted_center_df(main_truck)
                        main_truck = update_distance_matrix(main_truck, weighted_center_df)
                        print('init_cluster{} has been done'.format(init_cluster))
                        break
                if max(vol_lst) <= truck_capa:
                    break
            
        iteration += 1
        print('This iteration :', iteration)
        if iteration == 20:
            break
    print('LEVELING FINISHED')
    print(iteration)
    vol_lst = update_volume_lst(main_truck)
    vol_dict["Before_DBSCAN_Day_{:02d}".format(day)] = vol_lst
    main_truck.to_csv('0424_img/beforeDBSCAN_day_24.csv')
    
    #POST PROCESSING - DBSCAN
    X = list(zip(main_truck['x'].values, main_truck['y'].values))
    knn = NearestNeighbors(n_neighbors=6)
    knn.fit(X)
    predicted_df = pd.DataFrame()
    # del main_truck['outlier']
    for i in range(10):
        # print(i)
        smpl_df = main_truck[main_truck['reassign'] == i]
        data = smpl_df[['x', 'y']]

        mdl = DBSCAN(0.0015, min_samples=4)
        predict = pd.DataFrame(mdl.fit_predict(data))
        predict.columns=['outlier']
        
        smpl_df.reset_index(drop=True, inplace=True)
        smpl_df = pd.concat([smpl_df, predict['outlier']], axis=1)
        # smpl_df['outlier'] = predict['outlier']
        smpl_df['outlier'] = smpl_df['outlier'].apply(lambda x : 1 if x == -1 else 0)
        predicted_df = pd.concat([predicted_df, smpl_df], axis=0)

    main_truck = pd.merge(main_truck, predicted_df[['SPG_INNB', 'outlier']], how='left', on='SPG_INNB')
    outliers_df = main_truck[main_truck['outlier'] == 1]

    main_truck_co = main_truck.copy(deep=True) # DBSCAN으로 보정 전 정보를 활용해야 하기 때문에 deep copy
    
    for i in range(len(outliers_df)):
        outlier_p = outliers_df.iloc[i]
        temp_idx = outlier_p['index']
        temp_neighbor = knn.kneighbors(X, return_distance=False)[temp_idx][1:]
        
        if sum(main_truck_co.iloc[temp_neighbor]['reassign'] == main_truck_co.iloc[temp_idx]['reassign']) == len(temp_neighbor):
            print('Leave this node alone')
            pass
        elif sum(main_truck_co.iloc[temp_neighbor]['reassign'] == main_truck_co.iloc[temp_idx]['reassign']) < len(temp_neighbor):
            closest_neighbor_idx = temp_neighbor[0]
            closest_neighbor_cl = main_truck_co.iloc[closest_neighbor_idx]['reassign']

            if vol_lst[closest_neighbor_cl] + outlier_p['volume'] <= truck_capa:
                vol_lst[outlier_p['reassign']] -= outlier_p['volume']
                vol_lst[closest_neighbor_cl] += outlier_p['volume']
                main_truck.iloc[temp_idx, main_truck.columns.get_loc('reassign')] = closest_neighbor_cl
  
    vol_lst = update_volume_lst(main_truck)
    print(vol_lst)
    vol_dict["After_DBSCAN_Day_{:02d}".format(day)] = vol_lst
    # main_truck.to_csv('0304_img/Day_{:02d}.csv'.format(day))
    ## 전체 시각화
    for i in range(10):
        a = main_truck[main_truck['reassign'] == i]
        plt.scatter(a['x'], a['y'])
    plt.scatter(weighted_center_df['x'],weighted_center_df['y'], c= 'blue', marker= "D")
    plt.scatter(org_cens['x'], org_cens['y'], c='k', marker="D")
    for i in range(10):
        plt.annotate(str(i), (org_cens['x'][i], org_cens['y'][i]), (org_cens['x'][i]+0.0001, org_cens['y'][i]+0.0001), c='k')
        plt.annotate(str(i), (weighted_center_df['x'][i], weighted_center_df['y'][i]), (weighted_center_df['x'][i]+0.0001, weighted_center_df['y'][i]+0.0001), c='blue')
    # plt.show()
    plt.savefig('0429_img/Day_{:02d}'.format(day))
    plt.close()

[183, 182, 206, 182, 193, 185, 188, 192, 190, 187]
[341, 182, 288, 326, 193, 187, 188, 192, 316, 187]
0
2400
[341, 182, 288, 326, 193, 187, 188, 192, 316, 187]
[341, 182, 288, 326, 193, 187, 188, 192, 316, 187]
initial volume is [341, 182, 288, 326, 193, 187, 188, 192, 316, 187]

--------------------Current init cluster 0--------------------

case1-2
case2-1

Current Moving Point's ID: 1168000035030200
바뀐 클러스터는 9 기존 클러스터는 0

Hindrance: 0
[292, 182, 288, 326, 193, 187, 188, 192, 316, 236]
넘친경우, if
Target Volume: 51	 Current Cummulative Volume: 49
case1-2
case2-1

Current Moving Point's ID: 1168000035020300
바뀐 클러스터는 9 기존 클러스터는 0

Hindrance: 0
[263, 182, 288, 326, 193, 187, 188, 192, 316, 265]
넘친경우, if
Target Volume: 51	 Current Cummulative Volume: 78
init_cluster0 has been done

--------------------Current init cluster 1--------------------


Current Moving Point's ID: 1168000037030200
Current Moving Point's cluster: 3
Current Moving Point's Volume: 1
Target Volume: 29	 Current Cummulati

In [45]:
# for i in range(10):
#     a = main_truck[main_truck['reassign'] == i]
#     plt.scatter(a['x'], a['y'])
# plt.scatter(weighted_center_df['x'],weighted_center_df['y'], c= 'blue', marker= "D")
# plt.scatter(org_cens['x'], org_cens['y'], c='k', marker="D")
# for i in range(10):
#     plt.annotate(str(i), (org_cens['x'][i], org_cens['y'][i]), (org_cens['x'][i]+0.0001, org_cens['y'][i]+0.0001), c='k')
#     plt.annotate(str(i), (weighted_center_df['x'][i], weighted_center_df['y'][i]), (weighted_center_df['x'][i]+0.0001, weighted_center_df['y'][i]+0.0001), c='blue')
# plt.scatter(outliers_df['x'], outliers_df['y'], marker='^', c='red')
# plt.show()
for i in range(len(outliers_df)):
    outlier_p = outliers_df.iloc[i]
    temp_idx = outlier_p['index']
    temp_neighbor = knn.kneighbors(X, return_distance=False)[temp_idx][1:]

    if sum(main_truck_co.iloc[temp_neighbor]['reassign'] == main_truck_co.iloc[temp_idx]['reassign']) == len(temp_neighbor):
        print('Leave this node alone')
        pass
    elif sum(main_truck_co.iloc[temp_neighbor]['reassign'] == main_truck_co.iloc[temp_idx]['reassign']) < len(temp_neighbor):
        closest_neighbor_idx = temp_neighbor[0]
        closest_neighbor_cl = main_truck_co.iloc[closest_neighbor_idx]['reassign']
        print('elif')
        if vol_lst[closest_neighbor_cl] + outlier_p['volume'] <= truck_capa:
            vol_lst[outlier_p['reassign']] -= outlier_p['volume']
            vol_lst[closest_neighbor_cl] += outlier_p['volume']
            main_truck.iloc[temp_idx, main_truck.columns.get_loc('reassign')] = closest_neighbor_cl
            print('truck_capa')

elif
truck_capa
elif
elif
elif
Leave this node alone
Leave this node alone
Leave this node alone


In [17]:
iteration

8

In [19]:
volume_df = pd.DataFrame(vol_dict)
volume_df.to_csv('0429_img/volume_df.csv')

In [ ]:
import pandas as pd
'''
위의 리스트 day의 main_truck 다 불러와.
node의 ID & reassign값을 기반 value_count진행.
normalize 진행
x,y,reassign, normalize값을 토대로 시각화 진행 (interactive library such as bokeh, plotly)
'''
combined_df = pd.read_csv('./Information/opArea_pop_outputSegok.csv', index_col=0)
combined_df = combined_df[['SPG_INNB', 'x', 'y', 'reassign']]
combined_df[['probability','0','1','2','3','4','5','6','7','8','9']] = 0
combined_df.index = combined_df['SPG_INNB']
for day in [3,4,6,9,10,12,16,17,21,22,24,26,28,29,30]:
    main_truck = pd.read_csv('0304_img/Day_{:02d}.csv'.format(day))
    for idx, row in main_truck.iterrows():
        spg_innb = row['SPG_INNB']
        reassign_value = row['reassign']
        combined_df.loc[spg_innb, str(reassign_value)] += 1
combined_df['total'] = combined_df.iloc[:, 5:15].sum(axis=1)
# def find_max_index(row):
#     return row[[str(i) for i in range(10)]].argmax()
# combined_df['reassign'] = combined_df.apply(find_max_index, axis=1)
# combined_df.index = range(len(combined_df))
# for i in range(len(combined_df)):
#     row = combined_df.iloc[i]
#     cl = int(row['reassign']) 
#     probability = row[str(cl)] / (row['total'] + 0.0001)
#     combined_df.loc[i, 'probability'] = probability

In [ ]:
import plotly.express as px
for i in range(10):
    fig = px.scatter(combined_df, x='x', y='y', opacity=combined_df['{0}'.format(i)] / (combined_df['total'] + 0.0001),
                    text= round(combined_df['{0}'.format(i)] / (combined_df['total'] + 0.0001), 5), title = 'cluster_{}'.format(i))

    # 각각의 scatter plot을 추가합니다.
    # for i in range(1, 10):
    #     fig.add_traces(px.scatter(combined_df, x='x', y='y', opacity=combined_df['{0}'.format(i)] / (combined_df['total'] + 0.0001),
    #                                color_discrete_sequence=[px.colors.qualitative.Plotly[i]]))
    hover_text = ['x: {}<br>y: {}<br>probability: {:.5f}<br>SPG_INNB: {}'.format(x, y, p, SPG_INNB) for x, y, p, SPG_INNB in zip(combined_df['x'], combined_df['y'], combined_df['{0}'.format(i)] / (combined_df['total'] + 0.0001), combined_df['SPG_INNB'])]
    fig.update_traces(mode='markers', hovertemplate=hover_text)


    # fig.update_traces(mode='markers',
    #                   hovertemplate='x: %{x}<br>' +
    #                   'y: %{y}<br>'+
    #                   'probability: %{text}<br>')
    # fig.show()
    # 그래프 출력
    fig.write_html('0304_img/plotly_{0}_revised.html'.format(i))
